In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 78.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.4/423.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 3.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 8.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import ee
ee.Authenticate()

Enter verification code:  4/1AVHEtk6qA5lnHDprziP4EPOZSPyvlhDF17LRoygfXv-hzU0HH3xYLb6uUcU



Successfully saved authorization token.


In [3]:
from utils import *



2023-04-21 14:02:33.037 INFO    matplotlib.font_manager: generated new fontManager


In [4]:


ee.Initialize()

In [5]:
def getFeatureRaster(feature, exclusionRaster, targetRaster):
    excls = exclusionRaster.clip(feature)
    targ = targetRaster.clip(feature)

    return targ.multiply(excls)

In [6]:
def reduceToArea(image):
    areaDict = image.pixelArea().reduceRegion(ee.Reducer.sum(), scale=140, geometry=image.geometry())
    area = ee.Number(areaDict.get('area')).divide(1e6)
    name = image.get("name")
    newFeat = ee.Feature(image.geometry())
    newFeat = newFeat.set({'area':area, 'name':name})
    #newFeat = ee.Feature.setGeometry()
    return newFeat

In [10]:
# Scenario 1 - Max Exclusions


common = getExclusions()
wind = getExclusions("wind")
solar = getExclusions("solar")


#Scenario 1 - Max Exclusions
MaxWindExclusions = common + wind
MaxSolarExclusions = common + solar

maxExcl = {"wind":MaxWindExclusions, "solar": MaxSolarExclusions}

#Scenario 2 - Med Exclusions: 
MedWindExclusions = common + wind
MedWindExclusions.remove("Peatland")
MedWindExclusions.remove("Areas of Natural Beauty")

MedSolarExclusions = common + solar
MedSolarExclusions.remove("Peatland")
MedSolarExclusions.remove("Areas of Natural Beauty")

medExcl = {"wind":MedWindExclusions, "solar": MedSolarExclusions}

#Scenario 3 - Min Exclusions
MinWindExclusions = common + wind
MinWindExclusions.remove("Peatland")
MinWindExclusions.remove("Areas of Natural Beauty")
MinWindExclusions.remove("England National Parks")
MinWindExclusions.remove("Scotland Scenic Areas")
MinWindExclusions.remove("Wales National Park")

MinSolarExclusions = common + solar
MinSolarExclusions.remove("Peatland")
MinSolarExclusions.remove("Areas of Natural Beauty")
MinSolarExclusions.remove("England National Parks")
MinSolarExclusions.remove("Scotland Scenic Areas")
MinSolarExclusions.remove("Wales National Park")

minExcl = {"wind":MinWindExclusions, "solar": MinSolarExclusions}


paramsDict = {"min":minExcl, "med":medExcl, "max": maxExcl}


def getPowerLocal(geometry, exclusions, power):
    localPower = power.multiply(exclusions).clip(geometry)
    return localPower

exDict = getExclusionsDict()
resultsDict = {}
paramList = list(paramsDict.keys())


ladNameList = load_csv_list("local_authorities_name.csv")[1:]

LADVector= ladsVector()

for idx, exp in enumerate(paramList):
    exclDict = getExclusionsDict()
    windExc = paramsDict[exp]['wind']
    solarExc = paramsDict[exp]['solar']
    print(windExc)
    
    #windExclusionList = [exclDict[x] for x in windExc]
    #for ex in windExclusionList:                      
    
    names = []
    solarAreas = []
    windAreas = []
    
    
    for idx2, geomName in enumerate(ladNameList):
        print('trying', geomName)
        areaRast = ee.Image.constant(1)
        areaRast = areaRast.reproject(areaRast.projection().atScale(100)).pixelArea()

        activeFeat = LADVector.filter(f"LAD21NM == '{geomName}'").first()
        activeGeom = activeFeat.geometry()

        areaRast = areaRast.clip(activeGeom)


        def getTotalArea(feat):
            return areaRast.reduceRegion(ee.Reducer.sum(), feat, 100)



        for x in windExc:
            ex = exclDict[x]
            areaRast = areaRast.multiply(ex)
        areaRast = minimumMappingUnit(areaRast, 7)
        windArea = getTotalArea(activeGeom).getInfo()['area']
        print('wind', windArea)

        areaRast = ee.Image.constant(1)
        areaRast = areaRast.reproject(areaRast.projection().atScale(100)).pixelArea()

        task = ee.batch.Export.image.toDrive(**{
    'image': areaRast,
    'description': f'wind-{geomName}-{exp}',
    'folder':exp,
    'scale': 100,
    'region': activeGeom
})
        task.start()
    
        for x in solarExc:
            ex = exclDict[x]
            areaRast = areaRast.multiply(ex)
        areaRast = minimumMappingUnit(areaRast, 7)
        task = ee.batch.Export.image.toDrive(**{
    'image': areaRast,
    'description': f'solar-{geomName}-{exp}',
    'folder':exp,
    'scale': 100,
    'region': activeGeom
})
        task.start()
        
        solarArea = getTotalArea(activeGeom).getInfo()['area']
        print('solar', solarArea)
        names.append(geomName)
        solarAreas.append(solarArea)
        windAreas.append(windArea)
        print(geomName, 'wind power', windArea, 'solar power', solarArea)
        print(idx2, 'of', len(ladNameList), 'done')
        #except:
        #    names.append(geomName)
        #    solarAreas.append('failed')
        #    windAreas.append('failed')
        #    print(idx2, 'of', len(ladNameList), 'failed')

        resultsFrame = pd.DataFrame([names, solarAreas, windAreas]).T
        resultsFrame.columns = ['area name', 'solar power area', 'wind powe area']
        resultsFrame.to_csv(f'{exp}.csv')
    resultsDict[exp] = resultsFrame
    
        #print(getTotalArea(LADVector.first().geometry()).getInfo())
    
    
        
    


string = '''
for exp in paramList:
    
    areaRast = ee.Image.constant(1)
    windExc = paramsDict[exp]['wind']
    solarExc = paramsDict[exp]['solar']
    print(solarExc)
    assert windExc != solarExc

    windExclusions = [exDict[x] for x in windExc]
    solarExclusions = [exDict[x] for x in solarExc]

    windExclusions = compute_exclusions(windExclusions)
    solarExclusions = compute_exclusions(solarExclusions)

    featCol = constituenciesVector()



    solarAreaImgCol = ee.ImageCollection(featCol.map(lambda x: getFeatureRaster(x, solarExclusions, areaRast).set("name", ee.String(x.get('pcon19nm'))) ))
    windAreaImgCol = ee.ImageCollection(featCol.map(lambda x: getFeatureRaster(x, windExclusions, areaRast).set("name", ee.String(x.get('pcon19nm'))) ))
    
    #windAreaImgCol = windAreaImgCol.updateMask(solarAreaImgCol.gte(1))

    windAreaReduced = windAreaImgCol.map(reduceToArea)
    windarr = windAreaReduced.aggregate_array('area').getInfo()
    windNames = windAreaReduced.aggregate_array('name').getInfo()
    windPFrame = pd.DataFrame(windarr, index=windNames, columns=["Wind Area (KM^2)"])

    solarAreaReduced = solarAreaImgCol.map(reduceToArea)
    solarr = solarAreaReduced.aggregate_array('area').getInfo()
    solNames = solarAreaReduced.aggregate_array('name').getInfo()
    solPFrame = pd.DataFrame(solarr, index=solNames, columns=["Solar Area (KM^2)"])

    potentialFrame = solPFrame.join(windPFrame)
    assert solarr != windarr
    resultsDict[exp] = potentialFrame
'''

['Roads', 'Railway', 'Protected Areas', 'Woodlands', 'Cycle Paths Wind', 'Surface Water', 'England Cultural Sites', 'Scotland Cultural Sites', 'Parks and Green Space', 'Functional Sites', 'Wind Speed', 'Transmission Lines', 'Built Up Areas', 'Slope', 'Existing Onshore Wind', 'Existing Other Renewable']
trying Middlesbrough
wind 103709.77446959254
solar 253082.3505265778
Middlesbrough wind power 103709.77446959254 solar power 253082.3505265778
0 of 362 done
trying Redcar and Cleveland
wind 6930547.804970894
solar 17922826.64415593
Redcar and Cleveland wind power 6930547.804970894 solar power 17922826.64415593
1 of 362 done
trying Stockton-on-Tees
wind 7097500.851997168
solar 16683487.535885805
Stockton-on-Tees wind power 7097500.851997168 solar power 16683487.535885805
2 of 362 done
trying Darlington
wind 13682908.681108303
solar 24278036.975614667
Darlington wind power 13682908.681108303 solar power 24278036.975614667
3 of 362 done
trying Halton
wind 497707.7546453738
solar 821709.5681

EEException: User memory limit exceeded.

In [ ]:
    resultsFrame = pd.DataFrame([names, solarAreas, windAreas]).T
    resultsFrame.columns = ['area name', 'solar power area', 'wind powe area']


In [ ]:
resultsFrame

In [ ]:
solarExclusions

In [ ]:
break

In [ ]:
solarExcmin = paramsDict["min"]['solar']
solarExcmax = paramsDict["max"]['solar']
solExclMin = [exDict[x] for x in solarExcmin]
solExclMax = [exDict[x] for x in solarExcmax]

In [ ]:
solExclMin == solExclMax

In [ ]:
solarExcmax

In [ ]:
potentialFrame

In [ ]:
constVect.first().get('pcon19nm').getInfo()

In [ ]:
constVect.map(lambda x: getPowerLocal(x, solarExclusions, areaRast)).getInfo()

In [ ]:
solarr == windarr

In [ ]:
solarr 

In [ ]:
areaReduced.first().getInfo()